## DATA FROM THE WEB

In [ ]:
from bs4 import BeautifulSoup
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests as rq
import seaborn as sns
import re
sns.set_context('notebook')

# 02 - Data from the Web

## Deadline
Monday October 17, 2016 at 11:59PM

## Important Notes
* Make sure you push on GitHub your Notebook with all the cells already evaluated (i.e., you don't want your colleagues to generate 
unnecessary Web traffic during the peer review :)
* Don't forget to add a textual description of your thought process, the assumptions you made, and the solution
you plan to implement!
* Please write all your comments in English, and use meaningful variable names in your code

## Background
In this homework we will extract interesting information from [IS-Academia](http://is-academia.epfl.ch/page-6228.html), the educational
portal of EPFL. Specifically, we will focus on the part that allows [public access to academic data](http://is-academia.epfl.ch/publicaccess-Bachelor-Master).
The list of registered students by section and semester is not offered as a downloadable dataset, so you will have to find a way to scrape the
information we need. On [this form](http://isa.epfl.ch/imoniteur_ISAP/%21gedpublicreports.htm?ww_i_reportmodel=133685247) you can select
the data to download based on different criteria (e.g., year, semester, etc.)

You are not allowed to download manually all the tables -- rather you have to understand what parameters the server accepts, and
generate accordingly the HTTP requests. For this task, [Postman](https://www.getpostman.com) with the [Interceptor extension](https://www.getpostman.com/docs/capture)
can help you greatly. I recommend you to watch this [brief tutorial](https://www.youtube.com/watch?v=jBjXVrS8nXs&list=PLM-7VG-sgbtD8qBnGeQM5nvlpqB_ktaLZ&autoplay=1)
to understand quickly how to use it.
Your code in the iPython Notebook should not contain any hardcoded URL. To fetch the content from the IS-Academia server,
you can use the [Requests](http://docs.python-requests.org/en/master/) library with a Base URL, but all the other form parameters
should be extracted from the HTML with [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/).
You can choose to download Excel or HTML files -- they both have pros and cons, as you will find out after a quick check. You can also
choose to download data at different granularities (e.g., per semester, per year, etc.) but I recommend you not to download all the data
in one shot because 1) the requests are likely to timeout and 2) we will overload the IS-Academia server.

## Assignment
We will focus exclusively on the academic unit `Informatique`. When asked, select the right [statistical test](http://hamelg.blogspot.ch/2015/11/python-for-data-analysis-part-24.html).

1. Obtain all the data for the Bachelor students, starting from 2007. Keep only the students for which you have an entry for both `Bachelor
semestre 1` and `Bachelor semestre 6`. Compute how many months it took each student to go from the first to the sixth semester. Partition
the data between male and female students, and compute the average -- is the difference in average statistically significant?

2. Perform a similar operation to what described above, this time for Master students. Notice that this data is more tricky, as there are
many missing records in the IS-Academia database. Therefore, try to guess how much time a master student spent at EPFL by at least checking
the distance in months between `Master semestre 1` and `Master semestre 2`. If the `Mineur` field is *not* empty, the student should also
appear registered in `Master semestre 3`. Last but not the least, don't forget to check if the student has an entry also in the `Projet Master`
tables. Once you can handle well this data, compute the "average stay at EPFL" for master students. Now extract all the students with a 
`Spécialisation` and compute the "average stay" per each category of that attribute -- compared to the general average, can you find any
specialization for which the difference in average is statistically significant?

3. *BONUS*: perform the gender-based study also on the Master students, as explained in 1. Use scatterplots to visually identify changes
over time. Plot males and females with different colors -- can you spot different trends that match the results of your statistical tests?

## GET THE GOOD TEST (EXPLAIN)

## Function getParamsRequestValue

#### TODO comment

In [ ]:
def getParamsRequestValue(i_reportmodel_num, i_reportmodel_xsl_num, section, year, semester): 
    r = rq.get('http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter', params={'ww_i_reportmodel' : i_reportmodel_num})
    parameters_map = { 'ww_x_UNITE_ACAD' : section, 'ww_x_PERIODE_ACAD' : year, 'ww_x_PERIODE_PEDAGO' : semester }
    
    soup = BeautifulSoup(r.text, 'html.parser')
    
    ps = {key: next((option.attrs['value'] for option in soup.find('select', {'name': key }) if option.text==value), None) for key, value in parameters_map.items()}
    o = {'ww_i_reportmodel' : i_reportmodel_num, 'ww_b_list' : 1, 'ww_i_reportModelXsl' : i_reportmodel_xsl_num, 'ww_x_HIVERETE' : 'null', 'dummy' : 'ok'}
    d = {**o, **ps}
    r = rq.get('http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter', params=d)
    soup = BeautifulSoup(r.text, 'html.parser')
    for p in soup.find_all('a'):
        if(p.text==', '.join([section, year, semester])):
            arr = re.search('\(\'(.+?)\'\)', p['onclick']).group(1).split("=")
            d[arr[0]] = int(arr[1])
                  
    return d

#getParamsRequestValue(133685247, 133685270, 'Informatique', '2007-2008', 'Bachelor semestre 1')

### Initialize the value of reportmodel and Section, years and semesters we want.

In [ ]:
ww_i_reportmodel_num=133685247
ww_i_reportModelXsl_num=133685270
section = 'Informatique'
bachelor_semesters = ['Bachelor semestre 1', 'Bachelor semestre 2', 'Bachelor semestre 3', 'Bachelor semestre 4', 'Bachelor semestre 5', 'Bachelor semestre 6']
years_semesters = {'2007-2008': ['Bachelor semestre 1', 'Bachelor semestre 2'], '2008-2009': ['Bachelor semestre 1', 'Bachelor semestre 2', 'Bachelor semestre 3', 'Bachelor semestre 4'] , '2009-2010': bachelor_semesters, '2010-2011': bachelor_semesters, '2011-2012': bachelor_semesters, '2012-2013':bachelor_semesters, '2013-2014': bachelor_semesters, '2014-2015': bachelor_semesters, '2015-2016': ['Bachelor semestre 5', 'Bachelor semestre 6'], '2016-2017': ['Bachelor semestre 5', 'Bachelor semestre 6'] }

### Get columns' name

In [ ]:
list_attr = []
attributs = soup.find_all('th')
for attr in attributs:
    if(attr.string!=None):
        list_attr.append(attr.string)

list_attr.append("")
print(list_attr)

### Go through the years/semesters and get the data

In [ ]:
df_all = pd.DataFrame(columns=list_attr)
for year,semesters in years_semesters.items():
    for semester in semesters:
        payload = getParamsRequestValue(ww_i_reportmodel_num, ww_i_reportModelXsl_num, section, year, semester)
        r = rq.get('http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?', params=payload)
        soup = BeautifulSoup(r.text, 'html.parser')
        tds = soup.find_all('td')
        df = pd.DataFrame(columns=list_attr)
        pers = pd.Series()
        i=0
        j=0
        for elem in tds:
            pers.set_value(list_attr[(i%len(list_attr))], elem.string)
            i+=1
            if(i%len(list_attr)==0):
                j+=1
                df.loc[j]=pers
                pers.iloc[1:]

# We delete the empty column (and others if needed)
        del df[""]
        
        df["Bachelor Semestre"]=semester
        df_all = df_all.append(df, ignore_index=True)


In [ ]:
#df_all
df_all.to_csv('student_bachelor.csv', sep=';')

In [ ]:
# TEST FOR 1 year / 1 semester
semester="Bachelor semestre 1"
year="2015-2016"
payload = getParamsRequestValue(ww_i_reportmodel_num, ww_i_reportModelXsl_num, section, year, semester)
r = rq.get('http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?', params=payload)
soup = BeautifulSoup(r.text, 'html.parser')
#print(soup.prettify())

### Get the columns values of each student

In [ ]:
tds = soup.find_all('td')
df = pd.DataFrame(columns= list_attr)
pers = pd.Series()
i=0
j=0
for elem in tds:
    pers.set_value(list_attr[(i%12)], elem.string)
    i+=1
    if(i%12==0):
        j+=1
        df.loc[j]=pers
        pers.iloc[1:]

# We delete the empty column (and others if needed)
del df[""]

df["Bachelor Semestre"]=semester

#df

### We should create a csv with the datafram
df.to_csv(file_name, sep='\t', encoding='utf-8')

# Question 1
Obtain all the data for the Bachelor students, starting from 2007. Keep only the students for which you have an entry for both Bachelor
semestre 1 and Bachelor semestre 6. Compute how many months it took each student to go from the first to the sixth semester. Partition the data between male and female students, and compute the average -- is the difference in average statistically significant?